#### **1. Importing Libraries**

In [30]:
import pandas as pd                                                 # Importing for panel data analysis
#-------------------------------------------------------------------------------------------------------------------------------
import numpy as np                                                  # Importing package numpys (For Numerical Python)
#-------------------------------------------------------------------------------------------------------------------------------
import matplotlib.pyplot as plt                                     # Importing pyplot interface of matplotlib
import seaborn as sns                                               # Importing seaborn library for interactive visualization
%matplotlib inline
import random
import math
import time
import os
#--------------------~-----------------------------------------------------------------------------------------------------------
import pyfpgrowth                                                   # For testing the scratch implementation
#-------------------------------------------------------------------------------------------------------------------------------
import warnings                                                     # Importing warning to disable runtime warnings
warnings.filterwarnings("ignore")                                   # Warnings will appear only once

In [31]:
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
print('Shape of the dataset:', ratings.shape)
ratings.head(5)

Shape of the dataset: (100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [32]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [33]:
print("Number of unique users:", ratings["userId"].nunique())

Number of unique users: 610


In [34]:
print("Number of unique movies:", ratings["movieId"].nunique())

Number of unique movies: 9724


#### **2. Data preprocessing**

#### Form the transactional data set, which consists of entries of the form <user id, {movies rated above 2}

In [35]:
# let's extract the number of unique movies and its corresponding ratings
group = ratings.groupby('movieId')
df = group.apply(lambda x: x['rating'].unique())
df

movieId
1             [4.0, 4.5, 2.5, 3.5, 3.0, 5.0, 0.5, 2.0, 1.5]
2         [4.0, 3.0, 3.5, 4.5, 2.5, 5.0, 1.5, 1.0, 2.0, ...
3             [4.0, 5.0, 3.0, 3.5, 2.0, 1.0, 2.5, 0.5, 1.5]
4                                      [3.0, 1.0, 2.0, 1.5]
5             [5.0, 3.0, 4.0, 2.0, 3.5, 4.5, 1.5, 2.5, 0.5]
                                ...                        
193581                                                [4.0]
193583                                                [3.5]
193585                                                [3.5]
193587                                                [3.5]
193609                                                [4.0]
Length: 9724, dtype: object

- So, there are movies that have been rated 2 or less. Let's keep only entries where movie ratings are greater than 2. 

In [36]:
ratings_above_2 = ratings[ratings["rating"] > 2.0]
ratings_above_2

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [37]:
# Let's extract the number of unique movies that each user might have rated
group = ratings_above_2.groupby('userId')
df = group.apply(lambda x: len(x['movieId'].unique()))
df

userId
1       226
2        28
3        18
4       167
5        40
       ... 
606    1070
607     174
608     670
609      37
610    1233
Length: 610, dtype: int64

In [38]:
count_freq = dict(df)
count_freq

{1: 226,
 2: 28,
 3: 18,
 4: 167,
 5: 40,
 6: 294,
 7: 111,
 8: 43,
 9: 34,
 10: 119,
 11: 59,
 12: 32,
 13: 28,
 14: 42,
 15: 111,
 16: 96,
 17: 105,
 18: 493,
 19: 357,
 20: 210,
 21: 380,
 22: 70,
 23: 120,
 24: 107,
 25: 26,
 26: 19,
 27: 109,
 28: 476,
 29: 78,
 30: 34,
 31: 45,
 32: 99,
 33: 137,
 34: 67,
 35: 22,
 36: 35,
 37: 20,
 38: 63,
 39: 90,
 40: 94,
 41: 170,
 42: 353,
 43: 114,
 44: 38,
 45: 366,
 46: 42,
 47: 111,
 48: 33,
 49: 21,
 50: 236,
 51: 319,
 52: 130,
 53: 20,
 54: 31,
 55: 16,
 56: 46,
 57: 379,
 58: 103,
 59: 101,
 60: 22,
 61: 37,
 62: 357,
 63: 248,
 64: 504,
 65: 34,
 66: 337,
 67: 33,
 68: 1085,
 69: 44,
 70: 61,
 71: 30,
 72: 45,
 73: 187,
 74: 177,
 75: 51,
 76: 87,
 77: 25,
 78: 47,
 79: 60,
 80: 167,
 81: 17,
 82: 207,
 83: 95,
 84: 287,
 85: 27,
 86: 69,
 87: 20,
 88: 52,
 89: 425,
 90: 53,
 91: 495,
 92: 24,
 93: 97,
 94: 44,
 95: 160,
 96: 66,
 97: 35,
 98: 81,
 99: 44,
 100: 141,
 101: 50,
 102: 52,
 103: 362,
 104: 254,
 105: 717,
 106: 33,
 10

In [39]:
# create a new column to record the number of movies rated by each userId
ratings_above_2['count_freq_userId'] = ratings_above_2['userId']
ratings_above_2['count_freq_userId'] = ratings_above_2['count_freq_userId'].map(count_freq)
ratings_above_2

,userId,movieId,rating,timestamp,count_freq_userId
0,1,1,4.0,964982703,226
1,1,3,4.0,964981247,226
2,1,6,4.0,964982224,226
3,1,47,5.0,964983815,226
4,1,50,5.0,964982931,226
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,1233
100832,610,168248,5.0,1493850091,1233
100833,610,168250,5.0,1494273047,1233
100834,610,168252,5.0,1493846352,1233


----

- Let's keep only those users who have rated more than 10 movies.

In [40]:
# Now let's remove the rows where the value of 'count_freq_userId' is less than 10.
more_than_10_movies_rated_above_2 = ratings_above_2.drop(ratings_above_2[ratings_above_2['count_freq_userId'] <= 10].index)
print(more_than_10_movies_rated_above_2)

        userId  movieId  rating   timestamp  count_freq_userId
0            1        1     4.0   964982703                226
1            1        3     4.0   964981247                226
2            1        6     4.0   964982224                226
3            1       47     5.0   964983815                226
4            1       50     5.0   964982931                226
...        ...      ...     ...         ...                ...
100831     610   166534     4.0  1493848402               1233
100832     610   168248     5.0  1493850091               1233
100833     610   168250     5.0  1494273047               1233
100834     610   168252     5.0  1493846352               1233
100835     610   170875     3.0  1493846415               1233

[87295 rows x 5 columns]


----

- Let's create the transactional data of the form <user id, {movies rated above 2}>

In [41]:
# make a new dataframe with all unique userId
transactional_df = pd.DataFrame({'userId':more_than_10_movies_rated_above_2.userId.unique()})

# And then just get the list of all unique subreddits they are active in, assigning it to a new column
transactional_df['movies_rated_above_2'] = [set(more_than_10_movies_rated_above_2['movieId'].loc[more_than_10_movies_rated_above_2['userId'] == x['userId']]) 
    for _, x in transactional_df.iterrows()]

transactional_df

,userId,movies_rated_above_2
0,1,"{1024, 1, 1025, 3, 2048, 1029, 6, 1030, 1031, ..."
1,2,"{115713, 122882, 48516, 91529, 80906, 91658, 1..."
2,3,"{70946, 2851, 5764, 4518, 26409, 7991, 1275, 2..."
3,4,"{1025, 3079, 3083, 21, 1046, 2583, 4121, 538, ..."
4,5,"{1, 515, 261, 265, 527, 531, 21, 150, 534, 153..."
...,...,...
602,606,"{1, 8195, 6148, 7, 11, 69644, 4109, 15, 17, 18..."
603,607,"{1, 517, 2053, 2054, 1544, 3081, 11, 1036, 257..."
604,608,"{1, 4105, 10, 6157, 16, 21, 31, 32, 2080, 34, ..."
605,609,"{1, 137, 10, 650, 1161, 786, 150, 288, 161, 10..."


In [42]:
print("Number of unique users:", more_than_10_movies_rated_above_2["userId"].nunique())

Number of unique users: 607


In [43]:
print("Number of unique movies:", more_than_10_movies_rated_above_2["movieId"].nunique())

Number of unique movies: 8852


- As we observe, the number of unique users are reduced from 610 to 607 after preprocessing, and the number of unique movies have reduced from 9724 to 8852.

----

- Divide the data set into 80% training set and 20% test set. Remove 20% of
movies watched from each user and create a test set using the removed
movies

In [44]:
# dummy data
dummy_df = pd.DataFrame({'userId':[1,2,4,6,8], 'movies_rated_above_2':[[100,200,300,400,500,600,700], [100,200,300,400], [300,400,500,600,700,800], [500,600,700,800,900,1000,1100,1200], [700,800,900]]})
dummy_df

,userId,movies_rated_above_2
0,1,"[100, 200, 300, 400, 500, 600, 700]"
1,2,"[100, 200, 300, 400]"
2,4,"[300, 400, 500, 600, 700, 800]"
3,6,"[500, 600, 700, 800, 900, 1000, 1100, 1200]"
4,8,"[700, 800, 900]"


In [45]:
# dividing dummy df into 80-20 train-test, such that 20% of movies watched from each user is test set.
'''
- Parse through each user
- Randomly shuffle the items in the list and split into 80-20
- extract 20 of each user and make a separate df
'''

'\n- Parse through each user\n- Randomly shuffle the items in the list and split into 80-20\n- extract 20 of each user and make a separate df\n'

In [46]:
cols = ['userId', 'movies_rated_above_2']
train_df = pd.DataFrame(columns=cols)
test_df = pd.DataFrame(columns=cols)


# loop through the rows using iterrows()
for index, row in dummy_df.iterrows():
    # print(row['userId'], row['movies_rated_above_2'])
    print(row['movies_rated_above_2'])
    print("-----")
    n = int(np.ceil(0.2 * len(row['movies_rated_above_2'])))  # initialize a value that represents 20% of the total items in the list.
    test_list = random.sample(row['movies_rated_above_2'], n)  # randomly choose 20% of the values (n) from list and make a sublist.
    print("test_list", test_list)
    print("-----")
    train_list = [i for i in row['movies_rated_above_2'] if i not in test_list] # rest 80% values of list is in train/-list
    print("train_list", train_list) # randomly choose 20% of the values from list and make a sublist
    print("******************************************************")
    
    df_1 = pd.DataFrame({
    'userId': [row['userId']],
    'movies_rated_above_2': [train_list]
    })

    df_2 = pd.DataFrame({
    'userId': [row['userId']],
    'movies_rated_above_2': [test_list]
    })

    train_df = pd.concat([train_df, df_1])
    test_df = pd.concat([test_df, df_2])
    # print("index", index)
    # train_df.loc[index].userId = row['userId']
    # train_df.loc[index].movies_rated_above_2 = train_list

    # test_df.loc[index].userId = row['userId']
    # test_df.loc[index].movies_rated_above_2 = test_list


[100, 200, 300, 400, 500, 600, 700]
-----
test_list [400, 100]
-----
train_list [200, 300, 500, 600, 700]
******************************************************
[100, 200, 300, 400]
-----
test_list [300]
-----
train_list [100, 200, 400]
******************************************************
[300, 400, 500, 600, 700, 800]
-----
test_list [500, 400]
-----
train_list [300, 600, 700, 800]
******************************************************
[500, 600, 700, 800, 900, 1000, 1100, 1200]
-----
test_list [1100, 800]
-----
train_list [500, 600, 700, 900, 1000, 1200]
******************************************************
[700, 800, 900]
-----
test_list [900]
-----
train_list [700, 800]
******************************************************


In [47]:
dummy_df

,userId,movies_rated_above_2
0,1,"[100, 200, 300, 400, 500, 600, 700]"
1,2,"[100, 200, 300, 400]"
2,4,"[300, 400, 500, 600, 700, 800]"
3,6,"[500, 600, 700, 800, 900, 1000, 1100, 1200]"
4,8,"[700, 800, 900]"


In [48]:
train_df

,userId,movies_rated_above_2
0,1,"[200, 300, 500, 600, 700]"
0,2,"[100, 200, 400]"
0,4,"[300, 600, 700, 800]"
0,6,"[500, 600, 700, 900, 1000, 1200]"
0,8,"[700, 800]"


In [49]:
test_df

,userId,movies_rated_above_2
0,1,"[400, 100]"
0,2,[300]
0,4,"[500, 400]"
0,6,"[1100, 800]"
0,8,[900]


----

- Divide the data set into 80% training set and 20% test set. Remove 20% of
movies watched from each user and create a test set using the removed
movies

In [50]:
# extending the operations of dummy data on the original data

transactional_df

,userId,movies_rated_above_2
0,1,"{1024, 1, 1025, 3, 2048, 1029, 6, 1030, 1031, ..."
1,2,"{115713, 122882, 48516, 91529, 80906, 91658, 1..."
2,3,"{70946, 2851, 5764, 4518, 26409, 7991, 1275, 2..."
3,4,"{1025, 3079, 3083, 21, 1046, 2583, 4121, 538, ..."
4,5,"{1, 515, 261, 265, 527, 531, 21, 150, 534, 153..."
...,...,...
602,606,"{1, 8195, 6148, 7, 11, 69644, 4109, 15, 17, 18..."
603,607,"{1, 517, 2053, 2054, 1544, 3081, 11, 1036, 257..."
604,608,"{1, 4105, 10, 6157, 16, 21, 31, 32, 2080, 34, ..."
605,609,"{1, 137, 10, 650, 1161, 786, 150, 288, 161, 10..."


In [51]:
# dividing transactional_df df into 80-20 train-test, such that 20% of movies watched from each user is test set.
'''
- Parse through each user
- Randomly shuffle the items in the list and split into 80-20
- extract 20 of each user and make a separate df
'''

cols = ['userId', 'movies_rated_above_2']
train_df = pd.DataFrame(columns=cols)
test_df = pd.DataFrame(columns=cols)

# loop through the rows using iterrows()
for index, row in transactional_df.iterrows():
    # print(row['userId'], row['movies_rated_above_2'])
    # print(row['movies_rated_above_2'])
    # print("-----")
    n = int(np.ceil(0.2 * len(row['movies_rated_above_2']))) # initialize a value that represents 20% of the total items in the list.
    test_list = random.sample(row['movies_rated_above_2'], n)  # randomly choose 20% of the values (n) from list and make a sublist.
    # print("test_list", test_list)
    # print("-----")
    train_list = [i for i in row['movies_rated_above_2'] if i not in test_list] # rest 80% values of list is in train_list
    # print("train_list", train_list)
    # print("******************************************************")
    
    df_1 = pd.DataFrame({
    'userId': [row['userId']],
    'movies_rated_above_2': [train_list]
    })

    df_2 = pd.DataFrame({
    'userId': [row['userId']],
    'movies_rated_above_2': [test_list]
    })

    train_df = pd.concat([train_df, df_1])
    test_df = pd.concat([test_df, df_2])

In [52]:
train_df

,userId,movies_rated_above_2
0,1,"[1024, 1025, 3, 2048, 6, 1030, 1031, 1032, 205..."
0,2,"[115713, 122882, 48516, 80906, 91658, 77455, 7..."
0,3,"[70946, 2851, 5764, 4518, 7991, 2288, 849, 302..."
0,4,"[3079, 21, 1046, 2583, 4121, 538, 2076, 2078, ..."
0,5,"[1, 515, 265, 527, 531, 21, 150, 153, 410, 34,..."
...,...,...
0,606,"[1, 7, 4109, 15, 17, 18, 2073, 28, 29, 30749, ..."
0,607,"[1, 517, 2053, 2054, 1544, 3081, 11, 1036, 257..."
0,608,"[4105, 10, 6157, 16, 21, 31, 32, 2080, 2083, 4..."
0,609,"[137, 10, 650, 1161, 786, 150, 288, 1056, 1059..."


In [53]:
test_df

,userId,movies_rated_above_2
0,1,"[2470, 2640, 736, 2692, 1127, 235, 2826, 943, ..."
0,2,"[86345, 99114, 91529, 318, 131724, 112552]"
0,3,"[1587, 26409, 5919, 1275]"
0,4,"[1179, 759, 3996, 2203, 4021, 919, 1895, 3176,..."
0,5,"[457, 261, 290, 39, 534, 36, 296, 592]"
...,...,...
0,606,"[1244, 2962, 3855, 7569, 597, 7460, 6789, 1393..."
0,607,"[2403, 377, 3418, 2114, 1196, 2529, 434, 3440,..."
0,608,"[276, 1261, 2105, 1219, 2676, 1779, 3499, 480,..."
0,609,"[454, 161, 457, 292, 833, 1, 208, 329]"


In [54]:
# let's confirm if the first row of the transactional data has been split into 80-20.
print(len(transactional_df["movies_rated_above_2"].iloc[0]))
print(len(train_df["movies_rated_above_2"].iloc[0]))
print(len(test_df["movies_rated_above_2"].iloc[0]))

226
180
46


----

- Saving the 80% of training data and 20% of test data in the csv

In [55]:
# train_df.to_csv('./output/transactional_df_train.csv', index=False)
# test_df.to_csv('./output/transactional_df_test.csv', index=False)

----

#### **3. Association rule mining**

- From the training set, extract the set of all association rules of form X→Y, <br />
where X contains a single movie and Y contains the set of movies from the training set <br />
by employing the apriori or FPgrowth approach and set some minsup and minconf (eg : 50 and 0.1 respectively) <br />

In [56]:
# reading the training transactional data
# train_df = pd.read_csv('./output/transactional_df_train.csv')
# print('Shape of the dataset:', train_df.shape)
# train_df.head(5)

In [57]:
# reading the training transactional data
print('Shape of the dataset:', train_df.shape)
train_df.head(5)

Shape of the dataset: (607, 2)


,userId,movies_rated_above_2
0,1,"[1024, 1025, 3, 2048, 6, 1030, 1031, 1032, 205..."
0,2,"[115713, 122882, 48516, 80906, 91658, 77455, 7..."
0,3,"[70946, 2851, 5764, 4518, 7991, 2288, 849, 302..."
0,4,"[3079, 21, 1046, 2583, 4121, 538, 2076, 2078, ..."
0,5,"[1, 515, 265, 527, 531, 21, 150, 153, 410, 34,..."


In [58]:
# the type of the rows in the second column of the transactional dataframe 'train_df'
type(train_df.movies_rated_above_2.iloc[0])

list

In [59]:
train_df.movies_rated_above_2.iloc[0] 

[1024,
 1025,
 3,
 2048,
 6,
 1030,
 1031,
 1032,
 2054,
 527,
 1552,
 1042,
 2580,
 1049,
 543,
 3617,
 1060,
 1573,
 2596,
 552,
 553,
 1580,
 2093,
 2094,
 47,
 1073,
 50,
 2099,
 3639,
 1080,
 2105,
 2616,
 1089,
 1092,
 2116,
 70,
 2628,
 1097,
 3147,
 592,
 593,
 1617,
 596,
 1620,
 2641,
 2644,
 2648,
 1625,
 2137,
 2139,
 3671,
 2141,
 2654,
 2143,
 608,
 2657,
 101,
 1644,
 110,
 1136,
 2161,
 3702,
 3703,
 2174,
 2700,
 2193,
 3729,
 661,
 151,
 2716,
 157,
 3740,
 3744,
 673,
 163,
 3243,
 1196,
 1197,
 3253,
 1206,
 1210,
 1213,
 1214,
 1732,
 1222,
 1226,
 3273,
 3793,
 216,
 1240,
 2268,
 733,
 223,
 3809,
 1258,
 2797,
 1777,
 2291,
 1270,
 1275,
 1793,
 1282,
 260,
 1291,
 780,
 1804,
 1805,
 1298,
 2329,
 804,
 296,
 2858,
 2353,
 2872,
 3386,
 316,
 1348,
 333,
 2387,
 2899,
 2395,
 1377,
 356,
 362,
 2414,
 367,
 3439,
 3440,
 3441,
 1396,
 3448,
 3450,
 2427,
 1920,
 2944,
 2947,
 2948,
 2949,
 1927,
 2959,
 3479,
 923,
 2459,
 3489,
 1954,
 423,
 4006,
 2985,
 2987

In [60]:
train_df_key_value = dict(zip(train_df['userId'], train_df['movies_rated_above_2']))
train_df_key_value

{1: [1024,
  1025,
  3,
  2048,
  6,
  1030,
  1031,
  1032,
  2054,
  527,
  1552,
  1042,
  2580,
  1049,
  543,
  3617,
  1060,
  1573,
  2596,
  552,
  553,
  1580,
  2093,
  2094,
  47,
  1073,
  50,
  2099,
  3639,
  1080,
  2105,
  2616,
  1089,
  1092,
  2116,
  70,
  2628,
  1097,
  3147,
  592,
  593,
  1617,
  596,
  1620,
  2641,
  2644,
  2648,
  1625,
  2137,
  2139,
  3671,
  2141,
  2654,
  2143,
  608,
  2657,
  101,
  1644,
  110,
  1136,
  2161,
  3702,
  3703,
  2174,
  2700,
  2193,
  3729,
  661,
  151,
  2716,
  157,
  3740,
  3744,
  673,
  163,
  3243,
  1196,
  1197,
  3253,
  1206,
  1210,
  1213,
  1214,
  1732,
  1222,
  1226,
  3273,
  3793,
  216,
  1240,
  2268,
  733,
  223,
  3809,
  1258,
  2797,
  1777,
  2291,
  1270,
  1275,
  1793,
  1282,
  260,
  1291,
  780,
  1804,
  1805,
  1298,
  2329,
  804,
  296,
  2858,
  2353,
  2872,
  3386,
  316,
  1348,
  333,
  2387,
  2899,
  2395,
  1377,
  356,
  362,
  2414,
  367,
  3439,
  3440,
  3441,
  13

In [ ]:
test_df_key_value = dict(zip(test_df['userId'], test_df['movies_rated_above_2']))
test_df_key_value

In [61]:
train_df_key_value[1]

[1024,
 1025,
 3,
 2048,
 6,
 1030,
 1031,
 1032,
 2054,
 527,
 1552,
 1042,
 2580,
 1049,
 543,
 3617,
 1060,
 1573,
 2596,
 552,
 553,
 1580,
 2093,
 2094,
 47,
 1073,
 50,
 2099,
 3639,
 1080,
 2105,
 2616,
 1089,
 1092,
 2116,
 70,
 2628,
 1097,
 3147,
 592,
 593,
 1617,
 596,
 1620,
 2641,
 2644,
 2648,
 1625,
 2137,
 2139,
 3671,
 2141,
 2654,
 2143,
 608,
 2657,
 101,
 1644,
 110,
 1136,
 2161,
 3702,
 3703,
 2174,
 2700,
 2193,
 3729,
 661,
 151,
 2716,
 157,
 3740,
 3744,
 673,
 163,
 3243,
 1196,
 1197,
 3253,
 1206,
 1210,
 1213,
 1214,
 1732,
 1222,
 1226,
 3273,
 3793,
 216,
 1240,
 2268,
 733,
 223,
 3809,
 1258,
 2797,
 1777,
 2291,
 1270,
 1275,
 1793,
 1282,
 260,
 1291,
 780,
 1804,
 1805,
 1298,
 2329,
 804,
 296,
 2858,
 2353,
 2872,
 3386,
 316,
 1348,
 333,
 2387,
 2899,
 2395,
 1377,
 356,
 362,
 2414,
 367,
 3439,
 3440,
 3441,
 1396,
 3448,
 3450,
 2427,
 1920,
 2944,
 2947,
 2948,
 2949,
 1927,
 2959,
 3479,
 923,
 2459,
 3489,
 1954,
 423,
 4006,
 2985,
 2987

----

In [62]:
from collections import deque 

def traversetree(root):
    queue = deque([(root, root, 0)])
    while queue:
        parent_node, node, level = queue.popleft()
        print(f"{level = }")
        print(f"Parent: {parent_node.item}, Parent count: {parent_node.count}, Data: {node.item}, Count: {node.count}")
        for node_name in node.children:
            queue.append((node, node.children[node_name], level + 1))

def traverseheader(header_table):
    for key in header_table.keys():
        node = header_table[key]
        while node is not None:
            print(f"Header item: {key}, Link data: {node.item}, Link count: {node.count}")
            node = node.link 

In [63]:
#Global variable
id = 0
class Node:
    def __init__(self, item, count, parent):
        self.item = item           # Item value
        self.count = count         # Support count of the itemset
        self.parent = parent       # Parent node
        self.children = {}         # Children nodes (item: Node)
        self.link = None 

class FPGrowth:
    def __init__(self, data, minsup):
        self.data = data

    
    def find_frequent_items(self,data, minsup):
        header_table = {}
        for _, item_ls in data.items():
            for item in item_ls:
                header_table[item] = header_table.get(item, 0) + 1
        
        #Sort the dictionary
        # print(f"Before sorting {header_table = }")
        header_table = {k: v for k, v in sorted(header_table.items(), key=lambda item: (item[1], item[0]), reverse=True)}
        # print(f"After sorting {header_table = }")
        header_table = {k:-1 for k,v in header_table.items() if v>minsup}
        self.l = [*header_table.keys()]
        return header_table 
    
    #Constructing an FPTree
    def construct_fptree(self, data, header_table):
        root = Node(None,0,None)
        for _, transaction in data.items():
            ordered_transaction = [item for item in transaction if item in self.l]
            ordered_transaction.sort(key = lambda x:self.l.index(x))
            current_node = root
            # print(f"{ordered_transaction = }")
            for item in ordered_transaction:
                if item in current_node.children:
                    #Update the count of the already existing node
                    child_node = current_node.children[item]
                    child_node.count += 1
                else:
                    #Create a new node 
                    child_node = Node(item, 1, current_node)
                    current_node.children[item] = child_node
                    #Update header table
                    if item in header_table: #Why does this exist?
                        if header_table[item] == -1:
                            header_table[item] = child_node
                        else:
                            header_node = header_table[item]
                            while header_node.link is not None:
                                header_node =  header_node.link
                            header_node.link = child_node 
                current_node = child_node 
        return root, header_table

    #Mining an FPTree
    def mine_frequent_patterns(self, header_table, min_support, prefix=[]):
        global id
        frequent_patterns = []
        # Sort items in header table in descending order of frequency
        sorted_items = [item for item in header_table.keys()]
        sorted_items.sort(key=lambda x: (header_table[x].count, x))
        for item in sorted_items:
            new_prefix = prefix + [item]
            support = 0
            # Build the conditional pattern base
            conditional_dataset = {}
            node = header_table[item]
            while node is not None:
                count = node.count
                support += count 
                path = []
                current_node = node.parent
                while current_node.parent is not None:
                    path.append(current_node.item)
                    current_node = current_node.parent
                for _ in range(count):
                    conditional_dataset[id] = path
                    id += 1
                node = node.link
            frequent_patterns.append((new_prefix, support))
 
            
            # Recursively mine the conditional FP-tree
            conditional_header_table = self.find_frequent_items(conditional_dataset, min_support)
            root, conditional_header_table = self.construct_fptree(conditional_dataset, conditional_header_table)
            # print(f"Conditional prefix tree for prefix: {new_prefix}")
            # traversetree(root)
            # print()
            if conditional_header_table:
                frequent_patterns.extend(self.mine_frequent_patterns(conditional_header_table, min_support, new_prefix))
  
        return frequent_patterns
        

minsup = 10
FPGrowth_obj = FPGrowth(train_df_key_value, minsup)
header_table = FPGrowth_obj.find_frequent_items(train_df_key_value,minsup)
root, header_table = FPGrowth_obj.construct_fptree(train_df_key_value, header_table)
frequent_patterns = FPGrowth_obj.mine_frequent_patterns(header_table, minsup, [])
print(f"{frequent_patterns = }")
#For debugging
# traversetree(root)
# traverseheader(header_table)

: 

In [ ]:
#Testing with in-built python package
transactions = [['f', 'a', 'c', 'd', 'g', 'i', 'm', 'p'], ['a', 'b', 'c', 'f', 'l', 'm', 'o'],['b', 'f', 'h', 'j', 'o'], \
               ['b', 'c', 'k', 's', 'p'],['a', 'f', 'c', 'e', 'l', 'p', 'm', 'n']]
patterns = pyfpgrowth.find_frequent_patterns(transactions, 3)
print(f"{patterns = }")

NameError: name 'pyfpgrowth' is not defined